# Stateful Elasticsearch Feedback Workflow for Metrics Server
In this example we will add statistical performance metrics capabilities by levering the Seldon metrics server with persistence through the elasticsearch setup.

Dependencies
* Seldon Core installed
* Ingress provider (Istio or Ambassador)
* Install [Elasticsearch for the Seldon Core Logging](https://docs.seldon.io/projects/seldon-core/en/latest/analytics/logging.html)
* KNative eventing v0.11.0
* KNative serving v0.11.1 (optional)

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

Ambassador:

    kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080

Istio:

    kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80




In [76]:
%%writefile requirements-dev.txt
elasticsearch==7.9.1

Overwriting requirements-dev.txt


In [77]:
!pip install -r requirements-dev.txt

     |████████████████████████████████| 225kB 1.1MB/s eta 0:00:01
  Found existing installation: elasticsearch 7.5.1
    Uninstalling elasticsearch-7.5.1:
      Successfully uninstalled elasticsearch-7.5.1


In [5]:
!kubectl create namespace seldon || echo "namespace already created"

namespace/seldon created


In [6]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "docker-desktop" modified.


In [65]:
!mkdir -p config

Setting up Knative eventing routing for request logger 

In [37]:
!kubectl label namespace seldon-logs knative-eventing-injection=enabled --overwrite=true

namespace/seldon labeled


Adding payload request logger component for redirection of logs

In [17]:
%%bash
kubectl apply -f - << END
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
 name: seldon-request-logger-trigger
 namespace: seldon-logs
spec:
 subscriber:
   ref:
     apiVersion: v1
     kind: Service
     name: seldon-request-logger
END

kubectl apply -f - << END
apiVersion: apps/v1
kind: Deployment
metadata:
 name: seldon-request-logger
 namespace: seldon-logs
 labels:
   app: seldon-request-logger
spec:
 replicas: 1
 selector:
   matchLabels:
     app: seldon-request-logger
 template:
   metadata:
     labels:
       app: seldon-request-logger
   spec:
     containers:
       - name: user-container
         image: docker.io/seldonio/seldon-request-logger:1.4.0-dev
         imagePullPolicy: IfNotPresent
         env:
           - name: ELASTICSEARCH_HOST
             value: "elasticsearch-master.seldon-logs.svc.cluster.local"
           - name: ELASTICSEARCH_PORT
             value: "9200"
END

kubectl apply -f - << END
apiVersion: v1
kind: Service
metadata:
 name: seldon-request-logger
 namespace: seldon-logs
spec:
 selector:
   app: seldon-request-logger
 ports:
   - protocol: TCP
     port: 80
     targetPort: 8080
END

trigger.eventing.knative.dev/seldon-request-logger-trigger unchanged
deployment.apps/seldon-request-logger configured
service/seldon-request-logger unchanged


### Create a simple model
We create a multiclass classification model - iris classifier.

The iris classifier takes an input array, and returns the prediction of the 4 classes.

The prediction can be done as numeric or as a probability array.

In [132]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: multiclass-model
spec:
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
      logger:
        url: http://default-broker.seldon-logs.svc.cluster.local:80/
        mode: all
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/multiclass-model configured


In [18]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=multiclass-model -o jsonpath='{.items[0].metadata.name}')

deployment "multiclass-model-default-0-classifier" successfully rolled out


### Send Prediction Request

In [137]:
import requests
url = "http://localhost:80/seldon/seldon/multiclass-model/api/v1.0"

In [138]:
pred_req_1 = {"data":{"ndarray":[[1,2,3,4]]}}
pred_resp_1 = requests.post(f"{url}/predictions", json=pred_req_1)
print(pred_resp_1.json())
assert(len(pred_resp_1.json()["data"]["ndarray"][0])==3)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, 'meta': {}}


### Check data in Elasticsearch
We'll be able to check the elasticsearch through the service or the pods in our cluster.

In [139]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

See the indices that have been created

In [140]:
es.indices.get_alias("*")

{'inference-log-seldon-seldon-new-multiclass-model-default': {'aliases': {}},
 'inference-log-seldon-seldon-multiclass-model-shadow': {'aliases': {}},
 'inference-log-seldon-seldon-multiclass-model-default': {'aliases': {}},
 'inference-log-seldon-seldon-multiclass-model-canary': {'aliases': {}}}

In [141]:
puid_seldon_1 = pred_resp_1.headers.get("seldon-puid")

print(puid_seldon_1)

a1f84e43-41e8-4c92-a704-e2493a7f1c6e


Look at the data that is stored in the elasticsearch index

In [142]:
res = es.search(index="inference-log-seldon-seldon-multiclass-model-default", body={"query": {"match": { "_id": puid_seldon_1 }}})
print("Logged Request:")
print(res["hits"]["hits"][0]["_source"]["request"])
print("\nLogged Response:")
print(res["hits"]["hits"][0]["_source"]["response"])

Logged Request:
{'ce-source': 'http::8000', 'instance': [1.0, 2.0, 3.0, 4.0], 'payload': {'data': {'ndarray': [[1, 2, 3, 4]]}}, 'dataType': 'tabular', 'elements': {}, 'ce-time': '2020-11-06T08:53:24.964905828Z'}

Logged Response:
{'payload': {'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, 'meta': {}}, 'dataType': 'tabular', 'elements': {'t:0': [0.0006985194531162841], 't:1': [0.003668039039435755], 't:2': [0.9956334415074478]}, 'instance': [0.0006985194531162841, 0.003668039039435755, 0.9956334415074478], 'names': ['t:0', 't:1', 't:2'], 'ce-time': '2020-11-06T08:53:24.978390228Z', 'ce-source': 'http::8000'}


### Send feedback

We can now send the correction, or the truth value of the prediction.

For this we'll need to send the UUID of the feedback request to ensure it's added to the correct index.

We can also be able to add extra metadata, such as the user providing the feedback, date, time, etc.

In [143]:
feedback_tags_1 = {
    "user": "Seldon Admin",
    "date": "11/07/2020"
}

And finally we can put together the feedback request.

In [144]:
feedback_req_1 = {
    "reward": 0,
    "truth": {
        'data': {
            'names': ['t:0', 't:1', 't:2'], 
            'ndarray': [[0, 0, 1]]
        },
        "meta": {
            "tags": feedback_tags_1
        }
    }
}

And send the feedback request

In [145]:
feedback_resp_1 = requests.post(f"{url}/feedback", json=feedback_req_1, headers={"seldon-puid": puid_seldon_1})
print(feedback_resp_1)

<Response [200]>


Check that feedback has been received and stored in the Elasticsearch index

In [146]:
res = es.search(index="inference-log-seldon-seldon-multiclass-model-default", body={"query": {"match": { "_id": puid_seldon_1 }}})

print(res["hits"]["hits"][-1]["_source"]["feedback"])

{'reward': 0, 'ce-source': 'http::8000', 'truth': {'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[0, 0, 1]]}, 'meta': {'tags': {'date': '11/07/2020', 'user': 'Seldon Admin'}}}, 'ce-time': '2020-11-06T08:53:36.164873988Z'}


### Deploying Metrics Server

Now we'll be able to see how the metrics server makes use of this infrastructure patterns to provide real time performance metrics.

In [68]:
%%writefile config/multiclass-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: seldon-multiclass-model-metrics
  labels:
    app: seldon-multiclass-model-metrics
spec:
  replicas: 1
  selector:
    matchLabels:
      app: seldon-multiclass-model-metrics
  template:
    metadata:
      annotations:
        prometheus.io/path: /v1/metrics
        prometheus.io/scrape: "true"
      labels:
        app: seldon-multiclass-model-metrics
    spec:
      securityContext:
          runAsUser: 8888
      containers:
      - name: user-container
        image: seldonio/alibi-detect-server:1.4.0-dev
        imagePullPolicy: IfNotPresent
        args:
        - --model_name
        - multiclassserver
        - --http_port
        - '8080'
        - --protocol
        - seldonfeedback.http
        - --storage_uri
        - "adserver.cm_models.multiclass_one_hot.MulticlassOneHot"
        - --reply_url
        - http://message-dumper.default        
        - --event_type
        - io.seldon.serving.feedback.metrics
        - --event_source
        - io.seldon.serving.feedback
        - --elasticsearch_uri
        - http://elasticsearch-master.seldon-logs:9200
        - MetricsServer
        env:
        - name: "SELDON_DEPLOYMENT_ID"
          value: "multiclass-model"
        - name: "PREDICTIVE_UNIT_ID"
          value: "classifier"
        - name: "PREDICTIVE_UNIT_IMAGE"
          value: "alibi-detect-server:1.3.0-dev"
        - name: "PREDICTOR_ID"
          value: "default"
        ports:
        - containerPort: 8080
          name: metrics
          protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: seldon-multiclass-model-metrics
  labels:
    app: seldon-multiclass-model-metrics
spec:
  selector:
    app: seldon-multiclass-model-metrics
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8080

Overwriting config/multiclass-deployment.yaml


In [69]:
!kubectl apply -n seldon -f config/multiclass-deployment.yaml

deployment.apps/seldon-multiclass-model-metrics configured
service/seldon-multiclass-model-metrics unchanged


In [15]:
!kubectl rollout status deploy/seldon-multiclass-model-metrics

deployment "seldon-multiclass-model-metrics" successfully rolled out


### Trigger for metrics server

The trigger will be created in the seldon-logs namespace as that is where the initial trigger will be sent to.

In [52]:
%%bash

kubectl apply -f - << END
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
  name: multiclass-model-metrics-trigger
  namespace: seldon-logs
spec:
  filter:
    sourceAndType:
      type: io.seldon.serving.feedback
  subscriber:
    uri: http://seldon-multiclass-model-metrics.seldon:80
END

trigger.eventing.knative.dev/multiclass-model-metrics-trigger created


In [ ]:
import time
time.sleep(20)

### (Alternative) create kservice

If you want to create a kservice, and you've installed knative eventing and knative serving, you can use the instructions below.

The value of the file `config/multiclass-service.yaml` would be:
```
apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: seldon-multiclass-model-metrics
spec:
  template:
    metadata:
      annotations:
        prometheus.io/path: /v1/metrics
        prometheus.io/scrape: "true"
        autoscaling.knative.dev/minScale: "1"
    spec:
      containers:
      - image: "seldonio/alibi-detect-server:1.4.0-dev"
        args:
        - --model_name
        - multiclassserver
        - --http_port
        - '8080'
        - --protocol
        - seldonfeedback.http
        - --storage_uri
        - "adserver.cm_models.multiclass_one_hot.MulticlassOneHot"
        - --reply_url
        - http://message-dumper.default        
        - --event_type
        - io.seldon.serving.feedback.metrics
        - --event_source
        - io.seldon.serving.feedback
        - MetricsServer
        env:
        - name: "SELDON_DEPLOYMENT_ID"
          value: "multiclass-model"
        - name: "PREDICTIVE_UNIT_ID"
          value: "classifier"
        - name: "PREDICTIVE_UNIT_IMAGE"
          value: "alibi-detect-server:1.3.0-dev"
        - name: "PREDICTOR_ID"
          value: "default"
        ports:
        - containerPort: 8080
          name: metrics
          protocol: TCP
        securityContext:
            runAsUser: 8888
```

You can run the kservice with the command below:
```
kubectl apply -f config/multiclass-service.yaml
```
And then check with:

```
kubectl get kservice
```

You'll then have to create the trigger, first by creating the broker:

```
kubectl label namespace default knative-eventing-injection=enabled --overwrite=true
```

And then the trigger contents:

```
apiVersion: eventing.knative.dev/v1alpha1
kind: Trigger
metadata:
  name: multiclass-model-metrics-trigger
  namespace: default
spec:
  filter:
    sourceAndType:
      type: io.seldon.serving.feedback
  subscriber:
    ref:
      apiVersion: v1
      kind: Service
      name: seldon-multiclass-model-metrics
```

And you can run it with:
```
kubectl apply -f config/trigger.yaml
```

### Confirm empty metrics

In [71]:
!kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never -- \
    curl -v -X GET "http://seldon-multiclass-model-metrics.seldon.svc.cluster.local:80/v1/metrics" 

### Send Feedback

In [99]:
feedback_resp_1 = requests.post(f"{url}/feedback", json=feedback_req_1, headers={"seldon-puid": puid_seldon_1})
print(feedback_resp_1)

<Response [200]>


### Confirm metrics available

In [14]:
!kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never -- \
    curl -v -X GET "http://seldon-multiclass-model-metrics.seldon:80/v1/metrics" 

* Couldn't resolve host 'seldon-multiclass-model-metrics.seldon'
curl: (6) Couldn't resolve host 'seldon-multiclass-model-metrics.seldon'
pod seldon/curl terminated (Error)


## Forward Grafana Dashboard

Now we should be able to access the grafana dashboard for the model. You can access it by port-forwarding the grafana dashboard with:
```
kubectl port-forward -n seldon-system svc/seldon-core-analytics-grafana 7000:80
```

And you can load the following dashboard via the import tool:

In [ ]:
{"annotations": {"list": [{"builtIn": 1,"datasource": "-- Grafana --","enable": true,"hide": true,"iconColor": "rgba(0, 211, 255, 1)","name": "Annotations & Alerts","type": "dashboard"}]},"editable": true,"gnetId": null,"graphTooltip": 0,"id": 4,"links": [],"panels": [{"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {},"mappings": [],"thresholds": {"mode": "absolute","steps": [{"color": "green","value": null},{"color": "red","value": 80}]}},"overrides": []},"gridPos": {"h": 5,"w": 8,"x": 0,"y": 0},"id": 5,"options": {"colorMode": "background","graphMode": "area","justifyMode": "auto","orientation": "auto","reduceOptions": {"calcs": ["mean"],"fields": "","values": false}},"pluginVersion": "7.0.3","targets": [{"expr": "((sum(seldon_metric_true_positive_total) by (seldon_deployment_name) + sum(seldon_metric_true_negative_total) by (seldon_deployment_name))  /  (sum(seldon_metric_true_positive_total) by (seldon_deployment_name) + sum(seldon_metric_false_positive_total) by (seldon_deployment_name) + sum(seldon_metric_false_negative_total) by (seldon_deployment_name))) ","format": "time_series","instant": false,"interval": "","legendFormat": "{{class}} - {{seldon_app}}","refId": "A"}],"timeFrom": null,"timeShift": null,"title": "ACCURACY","transparent": true,"type": "stat"},{"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {},"mappings": [],"thresholds": {"mode": "absolute","steps": [{"color": "green","value": null},{"color": "red","value": 80}]}},"overrides": []},"gridPos": {"h": 5,"w": 8,"x": 8,"y": 0},"id": 6,"options": {"colorMode": "value","graphMode": "area","justifyMode": "auto","orientation": "auto","reduceOptions": {"calcs": ["mean"],"fields": "","values": false}},"pluginVersion": "7.0.3","targets": [{"expr": "(sum(seldon_metric_true_positive_total) by (seldon_deployment_name)  /  (sum(seldon_metric_true_positive_total) by (seldon_deployment_name) + sum(seldon_metric_false_positive_total) by (seldon_deployment_name))) ","instant": false,"interval": "","legendFormat": "{{class}} - {{seldon_deployment_name}}","refId": "A"}],"timeFrom": null,"timeShift": null,"title": "PRECISION","transparent": true,"type": "stat"},{"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {},"mappings": [],"thresholds": {"mode": "absolute","steps": [{"color": "green","value": null},{"color": "red","value": 80}]}},"overrides": []},"gridPos": {"h": 5,"w": 8,"x": 16,"y": 0},"id": 7,"options": {"colorMode": "value","graphMode": "area","justifyMode": "auto","orientation": "auto","reduceOptions": {"calcs": ["mean"],"fields": "","values": false}},"pluginVersion": "7.0.3","targets": [{"expr": "(sum(seldon_metric_true_positive_total) by (seldon_deployment_name)  /  (sum(seldon_metric_true_positive_total) by (seldon_deployment_name) + sum(seldon_metric_false_negative_total) by (seldon_deployment_name))) ","instant": false,"interval": "","legendFormat": "{{class}} - {{seldon_app}}","refId": "A"}],"timeFrom": null,"timeShift": null,"title": "Recall","transparent": true,"type": "stat"},{"aliasColors": {},"bars": false,"dashLength": 10,"dashes": false,"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {}},"overrides": []},"fill": 1,"fillGradient": 0,"gridPos": {"h": 6,"w": 24,"x": 0,"y": 5},"hiddenSeries": false,"id": 3,"legend": {"avg": false,"current": false,"max": false,"min": false,"show": true,"total": false,"values": false},"lines": true,"linewidth": 1,"nullPointMode": "null","options": {"dataLinks": []},"percentage": false,"pointradius": 2,"points": true,"renderer": "flot","seriesOverrides": [],"spaceLength": 10,"stack": false,"steppedLine": false,"targets": [{"expr": "((sum(seldon_metric_true_positive_total) by (class, seldon_deployment_name) + sum(seldon_metric_true_negative_total) by (class, seldon_deployment_name))  /  (sum(seldon_metric_true_positive_total) by (class, seldon_deployment_name) + sum(seldon_metric_false_positive_total) by (class, seldon_deployment_name) + sum(seldon_metric_false_negative_total) by (class, seldon_deployment_name))) ","interval": "","legendFormat": "{{class}} - {{seldon_deployment_name}}","refId": "A"}],"thresholds": [],"timeFrom": null,"timeRegions": [],"timeShift": null,"title": "Real Time Model ACCURACY by Class","tooltip": {"shared": true,"sort": 0,"value_type": "individual"},"transparent": true,"type": "graph","xaxis": {"buckets": null,"mode": "time","name": null,"show": true,"values": []},"yaxes": [{"decimals": null,"format": "short","label": null,"logBase": 1,"max": "1","min": "0","show": true},{"format": "short","label": null,"logBase": 1,"max": null,"min": null,"show": true}],"yaxis": {"align": false,"alignLevel": null}},{"aliasColors": {},"bars": false,"dashLength": 10,"dashes": false,"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {}},"overrides": []},"fill": 1,"fillGradient": 0,"gridPos": {"h": 7,"w": 12,"x": 0,"y": 11},"hiddenSeries": false,"id": 2,"legend": {"avg": false,"current": false,"max": false,"min": false,"show": true,"total": false,"values": false},"lines": true,"linewidth": 1,"nullPointMode": "null","options": {"dataLinks": []},"percentage": false,"pointradius": 2,"points": true,"renderer": "flot","seriesOverrides": [],"spaceLength": 10,"stack": false,"steppedLine": false,"targets": [{"expr": "((sum(seldon_metric_true_positive_total) by (class, seldon_deployment_name))  /  (sum(seldon_metric_true_positive_total) by (class, seldon_deployment_name) + sum(seldon_metric_false_positive_total) by (class, seldon_deployment_name))) ","interval": "","legendFormat": "{{class}} - {{seldon_deployment_name}}","refId": "A"}],"thresholds": [],"timeFrom": null,"timeRegions": [],"timeShift": null,"title": "Real Time Model PRECISION by Class","tooltip": {"shared": true,"sort": 0,"value_type": "individual"},"type": "graph","xaxis": {"buckets": null,"mode": "time","name": null,"show": true,"values": []},"yaxes": [{"decimals": null,"format": "short","label": null,"logBase": 1,"max": "1","min": "0","show": true},{"format": "short","label": null,"logBase": 1,"max": null,"min": null,"show": true}],"yaxis": {"align": false,"alignLevel": null}},{"aliasColors": {},"bars": false,"dashLength": 10,"dashes": false,"datasource": "prometheus","description": "","fieldConfig": {"defaults": {"custom": {}},"overrides": []},"fill": 1,"fillGradient": 0,"gridPos": {"h": 7,"w": 12,"x": 12,"y": 11},"hiddenSeries": false,"id": 4,"legend": {"avg": false,"current": false,"max": false,"min": false,"show": true,"total": false,"values": false},"lines": true,"linewidth": 1,"nullPointMode": "null","options": {"dataLinks": []},"percentage": false,"pointradius": 2,"points": true,"renderer": "flot","seriesOverrides": [],"spaceLength": 10,"stack": false,"steppedLine": false,"targets": [{"expr": "((sum(seldon_metric_true_negative_total) by (class, seldon_deployment_name))  /  (sum(seldon_metric_true_positive_total) by (class, seldon_deployment_name) + sum(seldon_metric_false_negative_total) by (class, seldon_deployment_name))) ","interval": "","legendFormat": "{{class}} - {{seldon_deployment_name}}","refId": "A"}],"thresholds": [],"timeFrom": null,"timeRegions": [],"timeShift": null,"title": "Real Time Model RECALL by Class","tooltip": {"shared": true,"sort": 0,"value_type": "individual"},"type": "graph","xaxis": {"buckets": null,"mode": "time","name": null,"show": true,"values": []},"yaxes": [{"decimals": null,"format": "short","label": null,"logBase": 1,"max": "1","min": "0","show": true},{"format": "short","label": null,"logBase": 1,"max": null,"min": null,"show": true}],"yaxis": {"align": false,"alignLevel": null}}],"refresh": false,"schemaVersion": 25,"style": "dark","tags": [],"templating": {"list": []},"time": {"from": "2020-11-03T12:06:17.311Z","to": "2020-11-03T13:06:17.315Z"},"timepicker": {"refresh_intervals": ["10s","30s","1m","5m","15m","30m","1h","2h","1d"]},"timezone": "","title": "Real Time Statistical Performance","uid": "St9vqHnGk","version": 1}

### Download prediction and feedback data

In [147]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np

X_test, y_test = datasets.load_iris(return_X_y=True)

# convert y to one hot
y_test = np.eye(y_test.max() + 1)[y_test]
print(y_test[:2])

[[1. 0. 0.]
 [1. 0. 0.]]


### Send Prediction Requests

In [148]:
import time

puids = []

for x in X_test:
    pred_req = {"data":{"ndarray":[x.tolist()]}}
    pred_resp = requests.post(f"{url}/predictions", json=pred_req)
    
    puid_seldon = pred_resp.headers.get("seldon-puid")
    puids.append(puid_seldon)
    time.sleep(0.1)

In [149]:
puids[:2]

['e10985c5-a032-4414-af1b-17473ca16e42',
 '884b98c9-c79c-401f-a5a3-9567f508e0f8']

In [150]:
for puid, y in zip(puids, y_test):
    data = {
        "truth": {
            'data': {
                'names': ['t:0', 't:1', 't:2'], 
                'ndarray': [y.tolist()]
            }
        }
    }
    requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
    time.sleep(0.2)

### Send randomized metrics to visualise degrade performance

In [35]:
y_rand = np.array(y_test)
np.random.shuffle(y_rand)
for puid, y in zip(puids, y_rand):
    data = {
        "truth": {
            'data': {
                'names': ['t:0', 't:1', 't:2'], 
                'ndarray': [y.tolist()]
            }
        }
    }
    requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
    time.sleep(0.5)

### VIsualise metrics

Now you should be able to see real time performance in the dashboard for "Accuracy", "Precision" and "Recall".

Furthermore you should also be able to get further insights from the data that is stored in Elasticsearch.

## Train model

#### Import data

In [190]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from alibi.explainers import AnchorTabular
from sklearn.model_selection import train_test_split

dataset = load_iris()
feature_names = dataset.feature_names
class_names = list(dataset.target_names)

#### Train our model

In [191]:
idx = 145
X_train,Y_train = dataset.data[:idx,:], dataset.target[:idx]
X_test, Y_test = dataset.data[idx+1:,:], dataset.target[idx+1:]

X = dataset.data
Y = dataset.target

np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=50)

In [192]:
idx = 0
print('Input: ', X_test[idx])
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Input:  [6.3 2.5 5.  1.9]
Prediction:  virginica


#### Export model for deployment

In [193]:
import joblib
with open("model.joblib", "wb") as file:
    joblib.dump(clf, file)

In [194]:
!ls | grep model

model.joblib


## Deploy model

In [195]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: multiclass-model
spec:
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
      logger:
        url: http://default-broker.seldon-logs.svc.cluster.local:80/
        mode: all
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/multiclass-model configured


In [196]:
!kubectl get pods | grep multiclass-model

multiclass-model-default-0-classifier-776746677-m5xr6   2/2     Running   0          12h
seldon-multiclass-model-metrics-57b6749597-d9dk2        1/1     Running   0          42h


### Send requests to visualise system performance

In [197]:
import time

while True:
    for x in X:
        pred_req = {"data":{"ndarray":[x.tolist()]}}

        pred_resp = requests.post(f"{url}/predictions", json=pred_req)

        time.sleep(0.1)

KeyboardInterrupt: 

### To send feedback first send requests and capture id

In [198]:
import time

puids = []

for x in X:
    pred_req = {"data":{"ndarray":[x.tolist()]}}
    pred_resp = requests.post(f"{url}/predictions", json=pred_req)
    
    puid_seldon = pred_resp.headers.get("seldon-puid")
    puids.append(puid_seldon)
    time.sleep(0.1)

#### Send feedback to visualise statistical performance (actual)

In [161]:
y_one_hot = np.eye(Y.max() + 1)[Y] # Covert to one hot
for puid, y in zip(puids, y_one_hot):
    data = {
        "truth": {
            'data': {
                'names': ['t:0', 't:1', 't:2'], 
                'ndarray': [y.tolist()]
            }
        }
    }
    requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
    time.sleep(0.5)

#### Send feedback to visualise statistical performance (Skewed)

In [200]:
y_one_hot = np.eye(Y.max() + 1)[Y] # Covert to one hot
y_rand = np.array(y_one_hot)
np.random.shuffle(y_rand)
while True:
    for puid, y in zip(puids, y_rand):
        data = {
            "truth": {
                'data': {
                    'names': ['t:0', 't:1', 't:2'], 
                    'ndarray': [y.tolist()]
                }
            }
        }
        requests.post(f"{url}/feedback", json=data, headers={"seldon-puid": puid})
        time.sleep(0.1)

KeyboardInterrupt: 

### Explainer
Now we can train and deploy an explainer.

#### Train explainer

In [201]:
predict_fn = lambda x: clf.predict_proba(x)

explainer = AnchorTabular(predict_fn, feature_names)

explainer.fit(X_train, disc_perc=(25, 50, 75))

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

#### Choose datapoint and prediction to explain

In [202]:
idx = 0
print('Input: ', X_test[idx])
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Input:  [6.3 2.5 5.  1.9]
Prediction:  virginica


#### Run explanation on datapoint

In [203]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.96
Coverage: 0.32


In [204]:
import dill
with open("explainer.dill", "wb") as file:
    dill.dump(explainer, file)

In [205]:
!ls | grep ex

explainer.dill


#### Deploy Explainer

In [206]:
%%bash

kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: iris-explainer
spec:
  annotations:
    seldon.io/rest-timeout: "100000"
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    explainer:
      type: AnchorTabular
      modelUri: gs://seldon-models/sklearn/iris-0.23.2/anchor
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/iris-explainer configured


#### Choose prediction to explain (the same one)

In [207]:
print('Input: ', X_test[idx])

Input:  [6.3 2.5 5.  1.9]


#### Send explanation request

In [208]:
explain_url = "http://localhost:80/seldon/seldon/iris-explainer-explainer/default/api/v1.0"

In [209]:
print(X_test[idx].tolist())

[6.3, 2.5, 5.0, 1.9]


In [210]:
pred_req_1 = {"data":{"ndarray":  [X_test[idx].tolist()] }}
explanation = requests.post(f"{explain_url}/explain", json=pred_req_1)
print(explanation)

<Response [200]>


In [211]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.98
Coverage: 0.32


In [98]:
!curl -X POST -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["text"], "ndarray": [[6.3, 2.5, 5.0, 1.9]]}}' \
    http://localhost:80/seldon/seldon/iris-explainer-explainer/default/api/v1.0/explain 

{"meta": {"name": "AnchorTabular", "type": ["blackbox"], "explanations": ["local"], "params": {"seed": null, "disc_perc": [25, 50, 75], "kwargs": {}, "verbose_every": 1, "verbose": false, "cache_margin": 1000, "binary_cache_size": 10000, "n_covered_ex": 10, "min_samples_start": 100, "max_anchor_size": null, "stop_on_first": false, "beam_size": 1, "coverage_samples": 10000, "batch_size": 100, "tau": 0.15, "delta": 0.1, "threshold": 0.95}}, "data": {"anchor": ["petal width (cm) > 1.80", "petal length (cm) > 4.20"], "precision": 0.9873873873873874, "coverage": 0.496551724137931, "raw": {"feature": [3, 2], "mean": [0.6406388642413487, 0.9873873873873874], "precision": [0.6406388642413487, 0.9873873873873874], "coverage": [0.20689655172413793, 0.496551724137931], "examples": [{"covered_true": [[5.1, 2.5, 3.0, 2.5], [6.0, 2.9, 4.5, 1.9], [6.0, 2.2, 4.0, 2.1], [5.7, 3.0, 4.2, 2.2], [5.9, 3.0, 4.2, 2.1], [5.4, 3.0, 4.5, 2.5], [7.6, 3.0, 6.6, 1.9], [6.8, 2.8, 4.8, 2.0], [6.0, 3.4, 4.5, 2.3], [7